In [1]:
! gdown 1yEjicXasMGFaALQobQWgsSxQgLEtxruC
! unzip /content/ThaiDepression.zip
! pip install --upgrade statsmodels

Downloading...
From: https://drive.google.com/uc?id=1yEjicXasMGFaALQobQWgsSxQgLEtxruC
To: /content/ThaiDepression.zip
100% 41.1k/41.1k [00:00<00:00, 71.0MB/s]
Archive:  /content/ThaiDepression.zip
  inflating: ThaiDepression/AHB_01.csv  
  inflating: ThaiDepression/AHB_02.csv  
  inflating: ThaiDepression/AHB_03.csv  
  inflating: ThaiDepression/AHB_04.csv  
  inflating: ThaiDepression/AHB_05.csv  
  inflating: ThaiDepression/AHB_06.csv  
  inflating: ThaiDepression/AHB_07.csv  
  inflating: ThaiDepression/AHB_08.csv  
  inflating: ThaiDepression/AHB_09.csv  
  inflating: ThaiDepression/AHB_10.csv  
  inflating: ThaiDepression/AHB_11.csv  
  inflating: ThaiDepression/AHB_12.csv  
  inflating: ThaiDepression/AHB_13.csv  
  inflating: ThaiDepression/All_AHB.csv  
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.8 MB 11.5 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: st

In [4]:
import pandas as pd
import numpy as np
import os
from numpy import array
from math import sqrt
from numpy import mean
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
import json
import plotly
import warnings
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import  plot_pacf
from statsmodels.tsa.stattools import pacf
import itertools
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import MinMaxScaler

#Unnormalized

In [50]:
file_list = os.listdir('ThaiDepression')
file_list.sort()
df_list = []
for i in range(14):
    df_list.append(pd.read_csv('ThaiDepression/' + file_list[i], index_col=0))
all_y = df_list[13].iloc[:,77:].T.reset_index(drop=True).T
all_y.index = pd.to_datetime(all_y.index)
exogs = []
for i in range(len(df_list)):
    if i == 13:
        exogs.append(df_list[i].iloc[:, :len(df_list[i].T)-14])      
    else:
        exogs.append(df_list[i].iloc[:, :len(df_list[i].T)-2])

In [51]:
warnings.filterwarnings("ignore")
all_pdq = []
for j in range(len(all_y.T)):
    sub_pdq = []
    # Find d 
    adf_res = adfuller(all_y[j], autolag = 'AIC')
    p_value = adf_res[1]
    d = 0
    while(p_value>=0.05):
        diff = all_y[j].diff(d+1)
        d = d + 1
        adf_res = adfuller(diff.dropna(), autolag = 'AIC')
        p_value = adf_res[1]


    # Find p 
    df_pacf = pacf(all_y[j].dropna(), method = 'ols')
    for i in range(0, len(df_pacf)):
        if df_pacf[i] < 1.96 / np.sqrt(len(all_y[j])):
            p = i - 1
            break


    sub_pdq.append(p)
    sub_pdq.append(d)


    # Find q
    aic = []
    possible_q = range(0, 50)

    for i in possible_q:
        try:
            model = ARIMA(all_y[j].dropna() , order = (p,d,i))
            results = model.fit()
            if len(aic)!=0:
                if results.aic > aic[-1]:
                    q = i - 1
                    break 
            aic.append(results.aic)
        except:
            continue


    sub_pdq.append(q)


    y_decompose = seasonal_decompose(all_y[j], model='additive',extrapolate_trend='freq')
    y_seasonal = y_decompose.seasonal
    # Find D
    adf_res = adfuller(y_seasonal, autolag = 'AIC')
    p_value = adf_res[1]
    D = 0
    while(p_value>=0.05):
        diff = y_seasonal.diff(d+1)
        D = D + 1
        adf_res = adfuller(diff.dropna(), autolag = 'AIC')
        p_value = adf_res[1]

    # Find P
    df_pacf = pacf(y_seasonal.dropna(), method = 'ols')
    for i in range(0, len(df_pacf)):
        if df_pacf[i] < 1.96 / np.sqrt(len(y_seasonal)):
            P = i - 1
            break


    sub_pdq.append(P)
    sub_pdq.append(D)


    # Find Q
    aic = []
    possible_Q = range(0, 50)

    for i in possible_Q:
        try:
            model = ARIMA(y_seasonal.dropna() , order = (P,D,i))
            results = model.fit()
            if len(aic)!=0:
                if results.aic > aic[-1]:
                    Q = i - 1
                    break 
            aic.append(results.aic)
        except:
            continue


    sub_pdq.append(Q)
    all_pdq.append(sub_pdq)    


warnings.filterwarnings("default")

In [52]:
train = all_y[0][:63]
test = all_y[0][63:]
M = [3, 4, 6, 9, 12]

performance_json = []
for i in range(len(all_pdq)):
    for j in range(len(M)):
        model_seasonal = SARIMAX(np.array(train), order=(all_pdq[i][0],all_pdq[i][1],all_pdq[i][2]), seasonal_order=(all_pdq[i][3],all_pdq[i][4],all_pdq[i][5],M[j]))
        model_fit_seasonal = model_seasonal.fit()
        ts_pred_seasonal = model_fit_seasonal.forecast(steps=7)
        mae = mean_absolute_error(test, ts_pred_seasonal)
        mse = mean_squared_error(test, ts_pred_seasonal)
        rmse = mean_squared_error(test, ts_pred_seasonal, squared = False)
        mape = mean_absolute_percentage_error(test, ts_pred_seasonal)

        model_seasonal = SARIMAX(np.array(train),np.array(exogs[i])[:63], order=(all_pdq[i][0],all_pdq[i][1],all_pdq[i][2]), seasonal_order=(all_pdq[i][3],all_pdq[i][4],all_pdq[i][5],M[j]))
        model_fit_seasonal = model_seasonal.fit()
        ts_pred_seasonal = model_fit_seasonal.forecast(steps=7, exog=np.array(exogs[i])[63:])
        m_mae = mean_absolute_error(test, ts_pred_seasonal)
        m_mse = mean_squared_error(test, ts_pred_seasonal)
        m_rmse = mean_squared_error(test, ts_pred_seasonal, squared = False)
        m_mape = mean_absolute_percentage_error(test, ts_pred_seasonal)

        performance_json.append({'model': 'SARIMAX', 'AHB_number': i+1, 'Multi_Uni': 'Uni', '[p,d,q,P,D,Q]': all_pdq[i], 'M': M[j],\
                                  'MAE': float(mae), 'MSE': float(mse), 'RMSE': float(rmse), 'MAPE': float(mape)})
        json_formatted_str = json.dumps(performance_json[-1], indent=2)
        print(json_formatted_str)

        performance_json.append({'model': 'SARIMAX', 'AHB_number': i+1, 'Multi_Uni': 'Multi', '[p,d,q,P,D,Q]': all_pdq[i], 'M': M[j],\
                                  'MAE': float(m_mae), 'MSE': float(m_mse), 'RMSE': float(m_rmse), 'MAPE': float(m_mape)})
        json_formatted_str = json.dumps(performance_json[-1], indent=2)
        print(json_formatted_str)

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


{
  "model": "SARIMAX",
  "AHB_number": 1,
  "Multi_Uni": "Uni",
  "[p,d,q,P,D,Q]": [
    1,
    1,
    1,
    0,
    0,
    0
  ],
  "M": 3,
  "MAE": 827.147340124473,
  "MSE": 839836.4000646807,
  "RMSE": 916.4258835632485,
  "MAPE": 0.005905624489769566
}
{
  "model": "SARIMAX",
  "AHB_number": 1,
  "Multi_Uni": "Multi",
  "[p,d,q,P,D,Q]": [
    1,
    1,
    1,
    0,
    0,
    0
  ],
  "M": 3,
  "MAE": 0.0,
  "MSE": 0.0,
  "RMSE": 0.0,
  "MAPE": 0.0
}


/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


{
  "model": "SARIMAX",
  "AHB_number": 1,
  "Multi_Uni": "Uni",
  "[p,d,q,P,D,Q]": [
    1,
    1,
    1,
    0,
    0,
    0
  ],
  "M": 4,
  "MAE": 827.147340124473,
  "MSE": 839836.4000646807,
  "RMSE": 916.4258835632485,
  "MAPE": 0.005905624489769566
}
{
  "model": "SARIMAX",
  "AHB_number": 1,
  "Multi_Uni": "Multi",
  "[p,d,q,P,D,Q]": [
    1,
    1,
    1,
    0,
    0,
    0
  ],
  "M": 4,
  "MAE": 0.0,
  "MSE": 0.0,
  "RMSE": 0.0,
  "MAPE": 0.0
}


/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


...

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


{
  "model": "SARIMAX",
  "AHB_number": 14,
  "Multi_Uni": "Uni",
  "[p,d,q,P,D,Q]": [
    1,
    1,
    1,
    0,
    0,
    1
  ],
  "M": 9,
  "MAE": 710.4815732468227,
  "MSE": 623283.322413074,
  "RMSE": 789.4829462458794,
  "MAPE": 0.005074492445269921
}
{
  "model": "SARIMAX",
  "AHB_number": 14,
  "Multi_Uni": "Multi",
  "[p,d,q,P,D,Q]": [
    1,
    1,
    1,
    0,
    0,
    1
  ],
  "M": 9,
  "MAE": 121.42710084932125,
  "MSE": 18493.642647560377,
  "RMSE": 135.99133298692377,
  "MAPE": 0.0008657286393062933
}


/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


{
  "model": "SARIMAX",
  "AHB_number": 14,
  "Multi_Uni": "Uni",
  "[p,d,q,P,D,Q]": [
    1,
    1,
    1,
    0,
    0,
    1
  ],
  "M": 12,
  "MAE": 913.0249876559038,
  "MSE": 1022513.5819606513,
  "RMSE": 1011.1941366328482,
  "MAPE": 0.006517575757830866
}
{
  "model": "SARIMAX",
  "AHB_number": 14,
  "Multi_Uni": "Multi",
  "[p,d,q,P,D,Q]": [
    1,
    1,
    1,
    0,
    0,
    1
  ],
  "M": 12,
  "MAE": 121.42710084932125,
  "MSE": 18493.642647560377,
  "RMSE": 135.99133298692377,
  "MAPE": 0.0008657286393062933
}


/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


In [53]:
name = "all_performance_SARIMAX_unnormalized"
json.dump(performance_json, open(name + ".json","w"))
df_json = pd.read_json('/content/' + name + '.json')
df_json.to_csv('/content/' + name + '.csv', index=False)
! cp /content/all_performance_SARIMAX_unnormalized.csv /content/drive/MyDrive/Depression

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ResourceWarning: unclosed file <_io.TextIOWrapper name='all_performance_SARIMAX_unnormalized.json' mode='w' encoding='UTF-8'>
  


#Normalized

In [5]:
file_list = os.listdir('ThaiDepression')
file_list.sort()
df_list = []
for i in range(14):
    df_list.append(pd.read_csv('ThaiDepression/' + file_list[i], index_col=0))

scalers = []
scalers_y = []
df_list_scale = []
for i in range(len(df_list)):
    if i == 13:
        scaler_y = MinMaxScaler()
        scaler_y.fit_transform(df_list[i].iloc[:, -1].to_numpy().reshape(len(df_list[i].iloc[:, -1]),-1))  
    else:
        scaler_y = MinMaxScaler()
        scaler_y.fit_transform(df_list[i].iloc[:, -2].to_numpy().reshape(len(df_list[i].iloc[:, -2]),-1))  
    scaler = MinMaxScaler()
    df_scale = pd.DataFrame(scaler.fit_transform(df_list[i]))
    df_scale.index = df_list[i].index
    df_scale.columns = df_list[i].columns
    scalers.append(scaler)
    df_list_scale.append(df_scale)
    scalers_y.append(scaler_y)

all_y = df_list_scale[13].iloc[:,77:].T.reset_index(drop=True).T
all_y.index = pd.to_datetime(all_y.index)
exogs = []
for i in range(len(df_list_scale)):
    if i == 13:
        exogs.append(df_list_scale[i].iloc[:, :len(df_list_scale[i].T)-14])      
    else:
        exogs.append(df_list_scale[i].iloc[:, :len(df_list_scale[i].T)-2])


In [6]:
warnings.filterwarnings("ignore")
all_pdq = []
for j in range(len(all_y.T)):
    sub_pdq = []
    # Find d 
    adf_res = adfuller(all_y[j], autolag = 'AIC')
    p_value = adf_res[1]
    d = 0
    while(p_value>=0.05):
        diff = all_y[j].diff(d+1)
        d = d + 1
        adf_res = adfuller(diff.dropna(), autolag = 'AIC')
        p_value = adf_res[1]


    # Find p 
    df_pacf = pacf(all_y[j].dropna(), method = 'ols')
    for i in range(0, len(df_pacf)):
        if df_pacf[i] < 1.96 / np.sqrt(len(all_y[j])):
            p = i - 1
            break


    sub_pdq.append(p)
    sub_pdq.append(d)


    # Find q
    aic = []
    possible_q = range(0, 50)

    for i in possible_q:
        try:
            model = ARIMA(all_y[j].dropna() , order = (p,d,i))
            results = model.fit()
            if len(aic)!=0:
                if results.aic > aic[-1]:
                    q = i - 1
                    break 
            aic.append(results.aic)
        except:
            continue


    sub_pdq.append(q)


    y_decompose = seasonal_decompose(all_y[j], model='additive',extrapolate_trend='freq')
    y_seasonal = y_decompose.seasonal
    # Find D
    adf_res = adfuller(y_seasonal, autolag = 'AIC')
    p_value = adf_res[1]
    D = 0
    while(p_value>=0.05):
        diff = y_seasonal.diff(d+1)
        D = D + 1
        adf_res = adfuller(diff.dropna(), autolag = 'AIC')
        p_value = adf_res[1]

    # Find P
    df_pacf = pacf(y_seasonal.dropna(), method = 'ols')
    for i in range(0, len(df_pacf)):
        if df_pacf[i] < 1.96 / np.sqrt(len(y_seasonal)):
            P = i - 1
            break


    sub_pdq.append(P)
    sub_pdq.append(D)


    # Find Q
    aic = []
    possible_Q = range(0, 50)

    for i in possible_Q:
        try:
            model = ARIMA(y_seasonal.dropna() , order = (P,D,i))
            results = model.fit()
            if len(aic)!=0:
                if results.aic > aic[-1]:
                    Q = i - 1
                    break 
            aic.append(results.aic)
        except:
            continue


    sub_pdq.append(Q)
    all_pdq.append(sub_pdq)    


warnings.filterwarnings("default")

In [18]:
train = all_y[0][:63]
test = all_y[0][63:]
M = [3, 4, 6, 9, 12]

performance_json = []
for i in range(len(all_pdq)):
    for j in range(len(M)):
        model_seasonal = SARIMAX(np.array(train), order=(all_pdq[i][0],all_pdq[i][1],all_pdq[i][2]), seasonal_order=(all_pdq[i][3],all_pdq[i][4],all_pdq[i][5],M[j]))
        model_fit_seasonal = model_seasonal.fit()
        ts_pred_seasonal = model_fit_seasonal.forecast(steps=7)
        mae = mean_absolute_error(test, ts_pred_seasonal)
        mse = mean_squared_error(test, ts_pred_seasonal)
        rmse = mean_squared_error(test, ts_pred_seasonal, squared = False)
        mape = mean_absolute_percentage_error(test, ts_pred_seasonal)
        ts_pred_seasonal_inverse = scalers_y[i].inverse_transform(ts_pred_seasonal.reshape(len(ts_pred_seasonal),-1))
        mae_inverse = mean_absolute_error(test, ts_pred_seasonal_inverse)
        mse_inverse = mean_squared_error(test, ts_pred_seasonal_inverse)
        rmse_inverse = mean_squared_error(test, ts_pred_seasonal_inverse, squared = False)
        mape_inverse = mean_absolute_percentage_error(test, ts_pred_seasonal_inverse)

        model_seasonal = SARIMAX(np.array(train),np.array(exogs[i])[:63], order=(all_pdq[i][0],all_pdq[i][1],all_pdq[i][2]), seasonal_order=(all_pdq[i][3],all_pdq[i][4],all_pdq[i][5],M[j]))
        model_fit_seasonal = model_seasonal.fit()
        ts_pred_seasonal = model_fit_seasonal.forecast(steps=7, exog=np.array(exogs[i])[63:])
        m_mae = mean_absolute_error(test, ts_pred_seasonal)
        m_mse = mean_squared_error(test, ts_pred_seasonal)
        m_rmse = mean_squared_error(test, ts_pred_seasonal, squared = False)
        m_mape = mean_absolute_percentage_error(test, ts_pred_seasonal)
        ts_pred_seasonal_inverse = scalers_y[i].inverse_transform(ts_pred_seasonal.reshape(len(ts_pred_seasonal),-1))
        m_mae_inverse = mean_absolute_error(test, ts_pred_seasonal_inverse)
        m_mse_inverse = mean_squared_error(test, ts_pred_seasonal_inverse)
        m_rmse_inverse = mean_squared_error(test, ts_pred_seasonal_inverse, squared = False)
        m_mape_inverse = mean_absolute_percentage_error(test, ts_pred_seasonal_inverse)

        performance_json.append({'model': 'SARIMAX', 'AHB_number': i+1, 'Multi_Uni': 'Uni', '[p,d,q,P,D,Q]': all_pdq[i], 'M': M[j],\
                                'MAE_normalize': float(mae), 'MSE_normalize': float(mse), 'RMSE_normalize': float(rmse), 'MAPE_normalize': float(mape), \
                                'MAE': float(mae_inverse), 'MSE': float(mse_inverse), 'RMSE': float(rmse_inverse), 'MAPE': float(mape_inverse)})
                                
        json_formatted_str = json.dumps(performance_json[-1], indent=2)
        print(json_formatted_str)

        performance_json.append({'model': 'SARIMAX', 'AHB_number': i+1, 'Multi_Uni': 'Multi', '[p,d,q,P,D,Q]': all_pdq[i], 'M': M[j],\
                                'MAE_normalize': float(m_mae), 'MSE_normalize': float(m_mse), 'RMSE_normalize': float(m_rmse), 'MAPE_normalize': float(m_mape), \
                                'MAE': float(m_mae_inverse), 'MSE': float(m_mse_inverse), 'RMSE': float(m_rmse_inverse), 'MAPE': float(m_mape_inverse)})
        json_formatted_str = json.dumps(performance_json[-1], indent=2)
        print(json_formatted_str)

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


{
  "model": "SARIMAX",
  "AHB_number": 1,
  "Multi_Uni": "Uni",
  "[p,d,q,P,D,Q]": [
    1,
    1,
    1,
    0,
    0,
    0
  ],
  "M": 3,
  "MAE_normalize": 0.006368645585721667,
  "MSE_normalize": 5.1017195921596115e-05,
  "RMSE_normalize": 0.007142632282400944,
  "MAPE_normalize": 0.00657572515032329,
  "MAE": 139784.22827732988,
  "MSE": 19545536126.93437,
  "RMSE": 139805.3508522988,
  "MAPE": 145557.96718119254
}
{
  "model": "SARIMAX",
  "AHB_number": 1,
  "Multi_Uni": "Multi",
  "[p,d,q,P,D,Q]": [
    1,
    1,
    1,
    0,
    0,
    0
  ],
  "M": 3,
  "MAE_normalize": 3.230096219069318e-08,
  "MSE_normalize": 1.2273297377852884e-15,
  "RMSE_normalize": 3.503326615925624e-08,
  "MAPE_normalize": 3.328274946614043e-08,
  "MAE": 139216.89371382832,
  "MSE": 19386308406.881065,
  "RMSE": 139234.72414193617,
  "MAPE": 144972.18546845755
}


/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


{
  "model": "SARIMAX",
  "AHB_number": 1,
  "Multi_Uni": "Uni",
  "[p,d,q,P,D,Q]": [
    1,
    1,
    1,
    0,
    0,
    0
  ],
  "M": 4,
  "MAE_normalize": 0.006368645585721667,
  "MSE_normalize": 5.1017195921596115e-05,
  "RMSE_normalize": 0.007142632282400944,
  "MAPE_normalize": 0.00657572515032329,
  "MAE": 139784.22827732988,
  "MSE": 19545536126.93437,
  "RMSE": 139805.3508522988,
  "MAPE": 145557.96718119254
}
{
  "model": "SARIMAX",
  "AHB_number": 1,
  "Multi_Uni": "Multi",
  "[p,d,q,P,D,Q]": [
    1,
    1,
    1,
    0,
    0,
    0
  ],
  "M": 4,
  "MAE_normalize": 3.230096219069318e-08,
  "MSE_normalize": 1.2273297377852884e-15,
  "RMSE_normalize": 3.503326615925624e-08,
  "MAPE_normalize": 3.328274946614043e-08,
  "MAE": 139216.89371382832,
  "MSE": 19386308406.881065,
  "RMSE": 139234.72414193617,
  "MAPE": 144972.18546845755
}


/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


...

{
  "model": "SARIMAX",
  "AHB_number": 14,
  "Multi_Uni": "Uni",
  "[p,d,q,P,D,Q]": [
    1,
    1,
    1,
    0,
    0,
    2
  ],
  "M": 6,
  "MAE_normalize": 0.006540767443462135,
  "MSE_normalize": 5.495953558411e-05,
  "RMSE_normalize": 0.007413469874769169,
  "MAPE_normalize": 0.00674744886007978,
  "MAE": 1229130.6914507635,
  "MSE": 1511076338557.6035,
  "RMSE": 1229258.4506756924,
  "MAPE": 1279998.0149922564
}
{
  "model": "SARIMAX",
  "AHB_number": 14,
  "Multi_Uni": "Multi",
  "[p,d,q,P,D,Q]": [
    1,
    1,
    1,
    0,
    0,
    2
  ],
  "M": 6,
  "MAE_normalize": 0.0009212564908379786,
  "MSE_normalize": 1.5754618028827582e-06,
  "RMSE_normalize": 0.0012551740129889393,
  "MAPE_normalize": 0.0009435904508404253,
  "MAE": 1225455.319865463,
  "MSE": 1502014141933.517,
  "RMSE": 1225566.8655497818,
  "MAPE": 1276199.965230662
}


/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/statespace/sarimax.py:868: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  ' zeros.' % warning_description)


{
  "model": "SARIMAX",
  "AHB_number": 14,
  "Multi_Uni": "Uni",
  "[p,d,q,P,D,Q]": [
    1,
    1,
    1,
    0,
    0,
    2
  ],
  "M": 9,
  "MAE_normalize": 0.006219182507771949,
  "MSE_normalize": 5.516316981411255e-05,
  "RMSE_normalize": 0.007427191246636412,
  "MAPE_normalize": 0.006390715557426148,
  "MAE": 1228923.9149097945,
  "MSE": 1510586509055.2522,
  "RMSE": 1229059.1967253864,
  "MAPE": 1279768.6383325167
}
{
  "model": "SARIMAX",
  "AHB_number": 14,
  "Multi_Uni": "Multi",
  "[p,d,q,P,D,Q]": [
    1,
    1,
    1,
    0,
    0,
    2
  ],
  "M": 9,
  "MAE_normalize": 0.000921256490837931,
  "MSE_normalize": 1.5754618028826654e-06,
  "RMSE_normalize": 0.0012551740129889024,
  "MAPE_normalize": 0.0009435904508403763,
  "MAE": 1225455.319865463,
  "MSE": 1502014141933.517,
  "RMSE": 1225566.8655497818,
  "MAPE": 1276199.9652306617
}


/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/statespace/sarimax.py:868: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  ' zeros.' % warning_description)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


{
  "model": "SARIMAX",
  "AHB_number": 14,
  "Multi_Uni": "Uni",
  "[p,d,q,P,D,Q]": [
    1,
    1,
    1,
    0,
    0,
    2
  ],
  "M": 12,
  "MAE_normalize": 0.0067263824609036715,
  "MSE_normalize": 5.673211975694315e-05,
  "RMSE_normalize": 0.007532072739753855,
  "MAPE_normalize": 0.006943555126399101,
  "MAE": 1229250.0404220584,
  "MSE": 1511367459817.3694,
  "RMSE": 1229376.8583381458,
  "MAPE": 1280124.1097526497
}
{
  "model": "SARIMAX",
  "AHB_number": 14,
  "Multi_Uni": "Multi",
  "[p,d,q,P,D,Q]": [
    1,
    1,
    1,
    0,
    0,
    2
  ],
  "M": 12,
  "MAE_normalize": 0.000921256490837931,
  "MSE_normalize": 1.5754618028826654e-06,
  "RMSE_normalize": 0.0012551740129889024,
  "MAPE_normalize": 0.0009435904508403763,
  "MAE": 1225455.319865463,
  "MSE": 1502014141933.517,
  "RMSE": 1225566.8655497818,
  "MAPE": 1276199.9652306617
}


In [ ]:
name = "all_performance_SARIMAX_normalized"
json.dump(performance_json, open(name + ".json","w"))
df_json = pd.read_json('/content/' + name + '.json')
df_json.to_csv('/content/' + name + '.csv', index=False)
! cp /content/all_performance_SARIMAX_normalized.csv /content/drive/MyDrive/Depression